# Credit Risk Analysis – Loan Default Prediction 💳📉

**Author:** Luis Chaumer  
**Role:** Data Analyst  

This project focuses on **credit risk analysis** for a fictional consumer lending portfolio.  
Using a synthetic dataset of **5,000 loans**, the goal is to:

- Explore drivers of loan default  
- Build a baseline **credit risk model** (classification)  
- Segment clients into **risk tiers**  
- Use both **Python and SQL** to analyze the portfolio  
- Provide insights and recommendations for risk management and lending policy  

The dataset used in this analysis is: `data/credit_risk_dataset.csv`.


## 1. Setup: imports and configuration

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report, RocCurveDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import sqlite3

plt.style.use("default")
pd.set_option("display.max_columns", 50)
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")

## 2. Load dataset

In [ ]:
data_path = "data/credit_risk_dataset.csv"

df = pd.read_csv(data_path, parse_dates=["application_date"])
df.head()

## 3. Initial exploration

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe().T

### 3.1 Target variable distribution (default)

In [ ]:
df["default"].value_counts(normalize=True).rename("proportion")

In [ ]:
plt.figure(figsize=(4,4))
df["default"].value_counts().plot(kind="bar")
plt.xticks([0, 1], ["No default (0)", "Default (1)"], rotation=0)
plt.title("Default distribution")
plt.ylabel("Number of loans")
plt.tight_layout()
plt.show()

## 4. Feature engineering

In [ ]:
df["year"] = df["application_date"].dt.year
df["month"] = df["application_date"].dt.month

df["income_bucket"] = pd.cut(
    df["annual_income"],
    bins=[0, 20000, 40000, 60000, 80000, 120000],
    labels=["very_low", "low", "medium", "high", "very_high"]
)

df["credit_bucket"] = pd.cut(
    df["credit_score"],
    bins=[499, 599, 659, 719, 850],
    labels=["poor", "fair", "good", "excellent"]
)

df.head()

## 5. Exploratory analysis

### 5.1 Correlation heatmap (numeric features)

In [ ]:
numeric_cols = ["age", "annual_income", "loan_amount", "term_months", "interest_rate", "credit_score", "dti", "default"]
corr = df[numeric_cols].corr()

plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, fmt=".2f")
plt.title("Correlation heatmap")
plt.tight_layout()
plt.show()

### 5.2 Default rate by credit bucket

In [ ]:
default_by_credit = df.groupby("credit_bucket")["default"].mean().rename("default_rate")
default_by_credit

In [ ]:
plt.figure(figsize=(6,4))
default_by_credit.plot(kind="bar")
plt.title("Default rate by credit score bucket")
plt.ylabel("Default rate")
plt.tight_layout()
plt.show()

### 5.3 Default rate by loan purpose

In [ ]:
default_by_purpose = df.groupby("loan_purpose")["default"].mean().sort_values(ascending=False)
default_by_purpose

In [ ]:
plt.figure(figsize=(8,4))
default_by_purpose.plot(kind="bar")
plt.title("Default rate by loan purpose")
plt.ylabel("Default rate")
plt.tight_layout()
plt.show()

## 6. SQL analysis with SQLite

In this section, we use **SQL** (via SQLite) to perform portfolio analysis tasks that are common in credit risk teams.

We will:

- Load the dataset into an in-memory SQLite database  
- Run several SQL queries to compute default rates and portfolio metrics  
- Show how Python and SQL can complement each other in a credit risk workflow  


In [ ]:
conn = sqlite3.connect(":memory:")
df.to_sql("loans", conn, index=False, if_exists="replace")

### 6.1 Default rate by loan purpose (SQL)

In [ ]:
query = '''
SELECT 
    loan_purpose,
    COUNT(*) AS total_loans,
    SUM(default) AS total_defaults,
    ROUND(AVG(default), 4) AS default_rate
FROM loans
GROUP BY loan_purpose
ORDER BY default_rate DESC;
'''
sql_default_by_purpose = pd.read_sql_query(query, conn)
sql_default_by_purpose

### 6.2 Default rate by credit bucket and income bucket (SQL)

In [ ]:
query = '''
SELECT 
    credit_bucket,
    income_bucket,
    COUNT(*) AS total_loans,
    SUM(default) AS total_defaults,
    ROUND(AVG(default), 4) AS default_rate
FROM loans
GROUP BY credit_bucket, income_bucket
ORDER BY credit_bucket, income_bucket;
'''
sql_segment_risk = pd.read_sql_query(query, conn)
sql_segment_risk

### 6.3 Portfolio metrics by year (SQL)

In [ ]:
query = '''
SELECT 
    year,
    COUNT(*) AS total_loans,
    ROUND(AVG(loan_amount), 2) AS avg_loan_amount,
    ROUND(AVG(interest_rate), 4) AS avg_interest_rate,
    ROUND(AVG(default), 4) AS default_rate
FROM loans
GROUP BY year
ORDER BY year;
'''
sql_by_year = pd.read_sql_query(query, conn)
sql_by_year

## 7. Modeling: baseline credit risk models

We build simple baseline models to predict **loan default**:

- **Logistic Regression** (interpretable baseline)  
- **Random Forest** (non-linear model, can capture interactions)  

This is not meant to be a full production-grade scorecard, but a demonstration of modeling workflow.


In [ ]:
features = [
    "age", "annual_income", "loan_amount", "term_months",
    "interest_rate", "credit_score", "dti"
]

X = df[features]
y = df["default"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

X_train.shape, X_test.shape

### 7.1 Logistic Regression

In [ ]:
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

y_pred_proba_lr = log_reg.predict_proba(X_test)[:, 1]
y_pred_lr = (y_pred_proba_lr >= 0.5).astype(int)

auc_lr = roc_auc_score(y_test, y_pred_proba_lr)
cm_lr = confusion_matrix(y_test, y_pred_lr)

auc_lr, cm_lr

In [ ]:
print("Logistic Regression classification report:")
print(classification_report(y_test, y_pred_lr))

In [ ]:
RocCurveDisplay.from_predictions(y_test, y_pred_proba_lr)
plt.title("ROC Curve - Logistic Regression")
plt.tight_layout()
plt.show()

### 7.2 Random Forest

In [ ]:
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=6,
    random_state=42
)
rf.fit(X_train, y_train)

y_pred_proba_rf = rf.predict_proba(X_test)[:, 1]
y_pred_rf = (y_pred_proba_rf >= 0.5).astype(int)

auc_rf = roc_auc_score(y_test, y_pred_proba_rf)
cm_rf = confusion_matrix(y_test, y_pred_rf)

auc_rf, cm_rf

In [ ]:
print("Random Forest classification report:")
print(classification_report(y_test, y_pred_rf))

In [ ]:
RocCurveDisplay.from_predictions(y_test, y_pred_proba_rf)
plt.title("ROC Curve - Random Forest")
plt.tight_layout()
plt.show()

### 7.3 Feature importance (Random Forest)

In [ ]:
importances = pd.Series(rf.feature_importances_, index=features).sort_values(ascending=False)
importances

In [ ]:
plt.figure(figsize=(6,4))
importances.plot(kind="bar")
plt.title("Feature importance - Random Forest")
plt.ylabel("Importance")
plt.tight_layout()
plt.show()

## 8. Risk segmentation

In [ ]:
df_test = X_test.copy()
df_test["default"] = y_test.values
df_test["pd_rf"] = y_pred_proba_rf

df_test["risk_tier"] = pd.cut(
    df_test["pd_rf"],
    bins=[0, 0.1, 0.25, 0.5, 1],
    labels=["very_low", "low", "medium", "high"]
)

risk_summary = df_test.groupby("risk_tier").agg(
    total_loans=("default", "count"),
    defaults=("default", "sum"),
    default_rate=("default", "mean"),
    avg_pd=("pd_rf", "mean")
)

risk_summary

## 9. Conclusions and recommendations

_Suggested narrative (to be adapted based on actual results):_

- The overall default rate in the portfolio is **X%**, with higher risk concentrated in lower credit score buckets and higher DTI segments.  
- **Credit score**, **DTI**, **interest rate** and **loan amount** emerge as key drivers of default risk.  
- The Random Forest model achieves an ROC-AUC of around **Y**, outperforming the logistic regression baseline and providing better rank-ordering of risk.  
- The risk tier segmentation (very_low / low / medium / high) clearly separates groups with different default rates, which can be used to adjust pricing, approval strategies or exposure limits.  
- SQL-based analysis proves useful for quickly summarizing portfolio risk by segments (purpose, year, credit bucket, income bucket).  

**Business recommendations:**

1. Tighten approval criteria or pricing in high-risk segments (e.g., low credit score + high DTI + small business loans).  
2. Use the risk tiers to define differentiated treatment strategies (e.g., monitoring, early collections, credit limits).  
3. Combine Python modeling with SQL reporting in a regular credit risk monitoring process.  
4. Extend the modeling with additional features (payment history, macro variables, bureau data) for production environments.  

This project demonstrates an end-to-end approach to **credit risk modeling** combining **Python, SQL, EDA, modeling and business interpretation**.
